In [ ]:
import requests
import io
import pandas as pd
import numpy as np
import talib
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Conv1D, Dropout, Concatenate
from keras.optimizers import Adam

# Download stock data
url = "https://alpha-vantage.p.rapidapi.com/query"
querystring = {
    "function": "TIME_SERIES_DAILY_ADJUSTED",
    "symbol": "MSFT",
    "outputsize": "full",
    "datatype": "csv",
}
headers = {
    "X-RapidAPI-Key": "YOUR_RAPIDAPI_KEY",
    "X-RapidAPI-Host": "YOUR_RAPIDAPI_HOST",
}
response = requests.request("GET", url, headers=headers, params=querystring)
df = pd.read_csv(io.BytesIO(response.content))

# Preprocess data
df['Date'] = pd.to_datetime(df['timestamp'])
df.set_index('Date', inplace=True)
df = df.sort_index(ascending=True)

# Generate technical indicators
df['SMA'] = talib.SMA(df['adjusted_close'], timeperiod=14)
df['EMA'] = talib.EMA(df['adjusted_close'], timeperiod=14)
df['RSI'] = talib.RSI(df['adjusted_close'], timeperiod=14)
df['MACD'], _, _ = talib.MACD(df['adjusted_close'], fastperiod=12, slowperiod=26, signalperiod=9)
df.dropna(inplace=True)

# News sentiment analysis
# Replace with your own dataset or API
df['News'] = np.random.randn(len(df))  # Dummy sentiment scores
df['Sentiment'] = df['News'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

# Feature scaling
scaler_prices = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler_prices.fit_transform(df[['adjusted_close', 'SMA', 'EMA', 'RSI', 'MACD', 'Sentiment']].values)

# Prepare LSTM and CNN inputs
lookback = 60
n_features = 6
X = []
y = []
for i in range(lookback, len(scaled_prices)):
    X.append(scaled_prices[i - lookback:i])
    y.append(scaled_prices[i, 0])  # Predict the 'adjusted_close' price

X, y = np.array(X), np.array(y)

# Split into training and test sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Define LSTM-CNN model
lstm_input = Input(shape=(lookback, n_features))
lstm = LSTM(units=50, return_sequences=True)(lstm_input)
lstm = Dropout(0.2)(lstm)
lstm = LSTM(units=50, return_sequences=True)(lstm)
lstm = Dropout(0.2)(lstm)
lstm = LSTM(units=50)(lstm)
lstm_output = Dropout(0.2)(lstm)

cnn_input = Input(shape=(lookback, n_features))
cnn = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_input)
cnn = Dropout(0.2)(cnn)
cnn = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn)
cnn_output = Dropout(0.2)(cnn)

merged = Concatenate()([lstm_output, cnn_output])
dense = Dense(units=64, activation='relu')(merged)
output = Dense(units=1, activation='linear')(dense)
model = Model(inputs=[lstm_input, cnn_input], outputs=output)

# Compile and fit model
model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')
model.fit([X_train, X_train], y_train, epochs=100, batch_size=32, validation_data=([X_test, X_test], y_test))

# Make predictions and inverse transform
predictions = model.predict([X_test, X_test])
predictions = scaler_prices.inverse_transform(np.hstack([predictions, np.zeros((predictions.shape[0], n_features - 1))]))[:, 0]